## 十四. 网页爬取(Web Scraper)

有些时候我们需要爬取一些 结构性比较强 的网页，并且需要将网页中的信息以JSON的方式返回回来。
我们就可以使用 LLMRequestsChain 类去实现，具体可以参考下面代码

为了方便理解，在例子中直接使用了Prompt的方法去格式化输出结果，而没用使用上个案例中用到的 StructuredOutputParser去格式化，也算是提供了另外一种格式化的思路

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

True

In [3]:
import os

from langchain.llms import AzureOpenAI
llm = AzureOpenAI(
    openai_api_base=os.getenv("AZURE_OPENAI_BASE_URL"),
    openai_api_version="2023-09-15-preview",
    deployment_name=os.getenv("AZURE_DEPLOYMENT_NAME_COMPLETE"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_type="azure",    
    #model_name="gpt-35-turbo",
)

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMRequestsChain, LLMChain

template = """在 >>> 和 <<< 之间是网页的返回的HTML内容。
网页是新浪财经A股上市公司的公司简介。
请抽取参数请求的信息。

>>> {requests_result} <<<
请使用如下的JSON格式返回数据
{{
  "company_name":"a",
  "company_english_name":"b",
  "issue_price":"c",
  "date_of_establishment":"d",
  "registered_capital":"e",
  "office_address":"f",
  "Company_profile":"g"

}}
Extracted:"""

prompt = PromptTemplate(
    input_variables=["requests_result"],
    template=template
)

chain = LLMRequestsChain(llm_chain=LLMChain(llm=llm, prompt=prompt))


In [6]:
inputs = {
  "url": "https://vip.stock.finance.sina.com.cn/corp/go.php/vCI_CorpInfo/stockid/600519.phtml"
}

response = chain(inputs)
print(response['output'])

 

{
  "company_name":"贵州茅台酒股份有限公司",
  "company_english_name":"Kweichow Moutai Co.,Ltd.",
  "issue_price":"31.39",
  "date_of_establishment":"1999-11-20",
  "registered_capital":"125620万元(CNY)",
  "office_address":"贵州省仁怀市茅台镇",
  "Company_profile":"    公司是根据贵州省人民政府黔府函〔1999〕291号文,由中国贵州茅台酒厂有限责任公司作为主发起人,联合贵州茅台酒厂技术开发公司、贵州省轻纺集体工业联社、深圳清华大学研究院、中国食品发酵工业研究院、北京市糖业烟酒公司、江苏省糖烟酒总公司、
